In [1]:
#Importing the important library that we will need in our project 
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split


In [2]:
# giving path of my data set 
DATA_PATH = r"C:\Users\HP\Downloads\UCI HAR Dataset/UCI HAR Dataset/"

In [3]:
# List of inertial signals to load
#all the signals where i am going to predict on raw signals 
INERTIAL_SIGNALS = ['body_acc_x', 'body_acc_y', 'body_acc_z', 'body_gyro_x', 'body_gyro_y', 'body_gyro_z'] 


In [4]:
"""Loads all the raw data from the dataset."""
#making a fuction 
def load_inertial_signals(folder, signals):  # folder - Specifies whether to load training ("train") or test ("test") data. signals :- raw data 
    
    data = [] # creating empty list :) 
    for signal in signals: # loop in signal file 
        file_path = f"{DATA_PATH}{folder}/Inertial Signals/{signal}_{folder}.txt" #this is my file path 
        signal_data = np.loadtxt(file_path) #this function will  load my file 
        data.append(signal_data) #append the data to signal file 
    return np.stack(data, axis=-1)   #converts the data list into a NumPy array with an extra dimension. axis =-1


In [9]:
#Load training and test data
X_train_raw = load_inertial_signals("train", INERTIAL_SIGNALS)
X_test_raw = load_inertial_signals("test", INERTIAL_SIGNALS)

y_train = np.loadtxt(DATA_PATH + "train/y_train.txt") - 1 # Loads the  labels from a text
y_test = np.loadtxt(DATA_PATH + "test/y_test.txt") - 1 # -1 is used -  Python we prefer 0-based indexing , labels are[1,4,2,3] - so [0,3,1,2]
#one-hot encoding use 0 based indexing 

In [13]:
y_train = to_categorical(y_train, num_classes=6)  
y_test = to_categorical(y_test, num_classes=6)

In [15]:
#Reads precomputed  features
X_train_features = np.loadtxt(DATA_PATH + "train/X_train.txt")
X_test_features = np.loadtxt(DATA_PATH + "test/X_test.txt")

In [19]:
#Normalize
X_train_features = (X_train_features - np.mean(X_train_features, axis=0)) / np.std(X_train_features, axis=0)
X_test_features = (X_test_features - np.mean(X_test_features, axis=0)) / np.std(X_test_features, axis=0)

In [41]:
#1D CNN for Raw Signals
#Extracts patterns 
#Reduces feature size.
#probabilities for 6 activity classes
model_raw = Sequential([
    Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(128, 6)),
    BatchNormalization(),
    MaxPooling1D(pool_size=2),
    
    Conv1D(filters=128, kernel_size=3, activation='relu'),
    BatchNormalization(),
    MaxPooling1D(pool_size=2),
    
    Conv1D(filters=256, kernel_size=3, activation='relu'),
    BatchNormalization(),
    MaxPooling1D(pool_size=2),
    
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(6, activation='softmax')
])


In [61]:

model_raw.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
#Train & Evaluate
model_raw.fit(X_train_raw, y_train, validation_data=(X_test_raw, y_test), epochs=40, batch_size=32)
test_loss_raw, test_acc_raw = model_raw.evaluate(X_test_raw, y_test)

Epoch 1/40
230/230 ━━━━━━━━━━━━━━━━━━━━ 5s 15ms/step - accuracy: 0.8742 - loss: 0.3217 - val_accuracy: 0.8191 - val_loss: 0.4977
Epoch 2/40
230/230 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - accuracy: 0.8949 - loss: 0.2823 - val_accuracy: 0.7699 - val_loss: 0.8257
Epoch 3/40
230/230 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - accuracy: 0.8813 - loss: 0.2846 - val_accuracy: 0.8103 - val_loss: 0.8051
Epoch 4/40
230/230 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - accuracy: 0.8822 - loss: 0.3449 - val_accuracy: 0.7258 - val_loss: 1.5216
Epoch 5/40
230/230 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.8941 - loss: 0.2707 - val_accuracy: 0.7991 - val_loss: 0.7657
Epoch 6/40
230/230 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - accuracy: 0.9053 - loss: 0.2779 - val_accuracy: 0.8904 - val_loss: 0.3694
Epoch 7/40
230/230 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - accuracy: 0.9181 - loss: 0.2184 - val_accuracy: 0.8945 - val_loss: 0.3150
Epoch 8/40
230/230 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - accuracy: 0.9118 - loss: 0.2405 - val_accu

In [63]:
#MLP for Predefined Features
model_features = Sequential([
    Dense(512, activation='relu', input_shape=(X_train_features.shape[1],)),
    Dropout(0.5),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(6, activation='softmax')
])

In [65]:
# Train & Evaluate the MLP Model
model_features.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [67]:
# Train & Evaluate the MLP Model
model_features.fit(X_train_features, y_train, validation_data=(X_test_features, y_test), epochs=40, batch_size=32)

Epoch 1/40
230/230 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.5709 - loss: 1.2257 - val_accuracy: 0.8694 - val_loss: 0.2621
Epoch 2/40
230/230 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8644 - loss: 0.3424 - val_accuracy: 0.9403 - val_loss: 0.1753
Epoch 3/40
230/230 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9159 - loss: 0.2153 - val_accuracy: 0.9376 - val_loss: 0.1698
Epoch 4/40
230/230 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9371 - loss: 0.1858 - val_accuracy: 0.9335 - val_loss: 0.1758
Epoch 5/40
230/230 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9517 - loss: 0.1405 - val_accuracy: 0.8928 - val_loss: 0.2615
Epoch 6/40
230/230 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9494 - loss: 0.1377 - val_accuracy: 0.9471 - val_loss: 0.1585
Epoch 7/40
230/230 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9583 - loss: 0.1167 - val_accuracy: 0.9406 - val_loss: 0.1730
Epoch 8/40
230/230 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9529 - loss: 0.1387 - val_accuracy: 0.

In [53]:
test_loss_features, test_acc_features = model_features.evaluate(X_test_features, y_test)

93/93 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9427 - loss: 0.1703


In [69]:
#test Accuracy on raw Signals 
print(f"Test Accuracy raw signals: {test_acc_raw * 100:}%")


Test Accuracy raw signals: 93.21343898773193%


In [71]:
#Test Accuracy on Predefined features 
print(f"Test Accuracy Predefined Features: {test_acc_features * 100:}%")

Test Accuracy Predefined Features: 94.7404146194458%


In [ ]:
#Accuracvy on Predefined features is better than Accuracy of model trained  on raw signal.